<a href="https://colab.research.google.com/github/sur-sakthy/computer-vision/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!cp '/content/drive/MyDrive/EE544 Computer Vision/imagewoof2-320.tgz' ./

In [ ]:
!tar zxvf /content/imagewoof2-320.tgz

In [38]:
import os
import cv2
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [22]:
PATH_TO_TRAIN_DATA = '/content/imagewoof2-320/train'
PATH_TO_VAL_DATA = '/content/imagewoof2-320/val'
IMG_WIDTH=224
IMG_HEIGHT=224

In [13]:
all_classes = os.listdir(PATH_TO_TRAIN_DATA)
all_classes

['n02105641',
 'n02087394',
 'n02096294',
 'n02115641',
 'n02088364',
 'n02086240',
 'n02099601',
 'n02089973',
 'n02093754',
 'n02111889']

In [14]:
all_classes = os.listdir(PATH_TO_VAL_DATA)
all_classes

['n02105641',
 'n02087394',
 'n02096294',
 'n02115641',
 'n02088364',
 'n02086240',
 'n02099601',
 'n02089973',
 'n02093754',
 'n02111889']

In [15]:
img_data_array = []
class_name = []

In [26]:
def load_dataset(img_folder): 
    for dir in os.listdir(img_folder):
      if dir == 'n02105641' or dir == 'n02087394' or dir == 'n02096294':
        for file in os.listdir(os.path.join(img_folder, dir)):
       
            image_path = os.path.join(img_folder, dir, file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir)

In [27]:
load_dataset(PATH_TO_TRAIN_DATA)
load_dataset(PATH_TO_VAL_DATA)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(img_data_array, class_name, test_size=0.30, random_state=42)

In [29]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.33, random_state=42)

In [60]:
def build_model():
  base_model = ResNet50(include_top=False, weights='imagenet')

  for layer in base_model.layers[:-33]: 
    layer.trainable = False

  # add a global spatial average pooling layer
  x = GlobalAveragePooling2D()(base_model.output)

  # add fully-connected
  x = Dense(512, activation='relu', name='fc-1')(x) 
  x = Dense(256, activation='relu', name='fc-2')(x)

  # a softmax layer for 3 classes
  out = Dense(3, activation='softmax', name='output_layer')(x)

  return Model(base_model.input, out)

In [61]:
custom_resnet_model = build_model()

In [77]:
for i in range(len(custom_resnet_model.layers)): 
  print(str(i) + ': ' + str(custom_resnet_model.layers[i].trainable))

0: False
1: False
2: False
3: False
4: False
5: False
6: False
7: False
8: False
9: False
10: False
11: False
12: False
13: False
14: False
15: False
16: False
17: False
18: False
19: False
20: False
21: False
22: False
23: False
24: False
25: False
26: False
27: False
28: False
29: False
30: False
31: False
32: False
33: False
34: False
35: False
36: False
37: False
38: False
39: False
40: False
41: False
42: False
43: False
44: False
45: False
46: False
47: False
48: False
49: False
50: False
51: False
52: False
53: False
54: False
55: False
56: False
57: False
58: False
59: False
60: False
61: False
62: False
63: False
64: False
65: False
66: False
67: False
68: False
69: False
70: False
71: False
72: False
73: False
74: False
75: False
76: False
77: False
78: False
79: False
80: False
81: False
82: False
83: False
84: False
85: False
86: False
87: False
88: False
89: False
90: False
91: False
92: False
93: False
94: False
95: False
96: False
97: False
98: False
99: False
100: False